# 09 特征可视化

> 理解 CNN "看到了什么"

---

## 📚 学习目标

- [ ] 可视化卷积核
- [ ] 可视化特征图
- [ ] 使用 Grad-CAM 解释模型决策

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import torch
    import torch.nn as nn
    import torchvision.models as models
    import torchvision.transforms as transforms
    from PIL import Image
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("请安装 PyTorch")

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

---

## 💻 1. 可视化卷积核

In [ ]:
if HAS_TORCH:
    # 加载预训练模型
    model = models.vgg16(weights='DEFAULT')
    
    # 获取第一层卷积核
    # VGG16 的第一层: 3 输入通道 → 64 输出通道, 3×3 核
    first_conv = model.features[0]
    kernels = first_conv.weight.data.cpu().numpy()
    
    print(f"第一层卷积核形状: {kernels.shape}")
    print(f"  = (64 个核, 3 通道, 3×3)")
    
    # 可视化前 16 个核（取 RGB 平均）
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    
    for i, ax in enumerate(axes.flatten()):
        if i < 16:
            kernel = kernels[i]
            # 归一化到 [0, 1]
            kernel = (kernel - kernel.min()) / (kernel.max() - kernel.min())
            # 转置为 (H, W, C)
            kernel = np.transpose(kernel, (1, 2, 0))
            ax.imshow(kernel)
            ax.set_title(f'核 {i}')
        ax.axis('off')
    
    plt.suptitle('VGG16 第一层卷积核可视化', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print("\n💡 第一层核通常检测边缘、颜色等基础特征")

---

## 💻 2. 可视化特征图

In [ ]:
if HAS_TORCH:
    # 创建一个简单的钩子来捕获特征图
    activation = {}
    
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    
    # 注册钩子
    model.features[0].register_forward_hook(get_activation('conv1'))
    model.features[10].register_forward_hook(get_activation('conv5'))
    model.features[24].register_forward_hook(get_activation('conv10'))
    
    # 创建一个随机输入
    dummy_input = torch.randn(1, 3, 224, 224)
    
    # 前向传播
    model.eval()
    with torch.no_grad():
        _ = model(dummy_input)
    
    # 可视化不同层的特征图
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    for ax, (name, feat) in zip(axes, activation.items()):
        # 取第一个样本，第一个通道
        feature_map = feat[0, 0].numpy()
        ax.imshow(feature_map, cmap='viridis')
        ax.set_title(f'{name}\n形状: {list(feat.shape)}')
        ax.axis('off')
    
    plt.suptitle('不同深度的特征图', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    print("💡 注意特征图尺寸随深度减小，而抽象程度增加")

---

## 💻 3. Grad-CAM 类激活图

Grad-CAM 显示模型在做决策时"关注"图像的哪些区域。

In [ ]:
if HAS_TORCH:
    class GradCAM:
        """
        Grad-CAM 实现
        
        公式:
            1. α_k = GAP(∂y_c/∂A_k)  (梯度的全局平均池化)
            2. L_cam = ReLU(Σ_k α_k · A_k)  (加权和)
        """
        
        def __init__(self, model, target_layer):
            self.model = model
            self.target_layer = target_layer
            self.gradients = None
            self.activations = None
            
            # 注册钩子
            target_layer.register_forward_hook(self.save_activation)
            target_layer.register_full_backward_hook(self.save_gradient)
        
        def save_activation(self, module, input, output):
            self.activations = output
        
        def save_gradient(self, module, grad_input, grad_output):
            self.gradients = grad_output[0]
        
        def __call__(self, x, class_idx=None):
            # 前向传播
            output = self.model(x)
            
            if class_idx is None:
                class_idx = output.argmax(dim=1).item()
            
            # 反向传播（对特定类别）
            self.model.zero_grad()
            one_hot = torch.zeros_like(output)
            one_hot[0, class_idx] = 1
            output.backward(gradient=one_hot, retain_graph=True)
            
            # 计算 Grad-CAM
            gradients = self.gradients.cpu().data.numpy()[0]  # (C, H, W)
            activations = self.activations.cpu().data.numpy()[0]  # (C, H, W)
            
            # 权重 = 梯度的全局平均池化
            weights = np.mean(gradients, axis=(1, 2))  # (C,)
            
            # 加权和
            cam = np.zeros(activations.shape[1:], dtype=np.float32)
            for i, w in enumerate(weights):
                cam += w * activations[i]
            
            # ReLU
            cam = np.maximum(cam, 0)
            
            # 归一化到 [0, 1]
            cam = cam - cam.min()
            cam = cam / cam.max()
            
            return cam, class_idx
    
    print("GradCAM 类已定义")
    print("\n使用方法:")
    print("  grad_cam = GradCAM(model, model.features[-1])")
    print("  cam, class_idx = grad_cam(image_tensor)")

In [ ]:
if HAS_TORCH:
    # 演示 Grad-CAM（使用随机图像）
    model = models.resnet18(weights='DEFAULT')
    model.eval()
    
    # 创建 GradCAM
    grad_cam = GradCAM(model, model.layer4)
    
    # 随机输入
    x = torch.randn(1, 3, 224, 224)
    x.requires_grad = True
    
    # 计算 CAM
    cam, class_idx = grad_cam(x)
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # 原图（取前3通道）
    img = x[0].detach().numpy().transpose(1, 2, 0)
    img = (img - img.min()) / (img.max() - img.min())
    axes[0].imshow(img)
    axes[0].set_title('输入图像')
    axes[0].axis('off')
    
    # CAM 热力图
    import cv2
    cam_resized = cv2.resize(cam, (224, 224))
    axes[1].imshow(img)
    axes[1].imshow(cam_resized, alpha=0.5, cmap='jet')
    axes[1].set_title(f'Grad-CAM (预测类别: {class_idx})')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 红色区域是模型做决策时关注的区域")

---

## 📝 本章小结

### 可视化技术对比

| 技术 | 目的 | 方法 |
|------|------|------|
| 卷积核可视化 | 看学到了什么滤波器 | 直接显示权重 |
| 特征图可视化 | 看特征表示 | 显示中间层输出 |
| Grad-CAM | 解释决策 | 梯度加权激活 |

### 关键洞察

- 浅层特征：边缘、颜色、纹理
- 深层特征：形状、部件、语义

---

## 🎯 完成 4.1 CNN 模块！

恭喜完成 CNN 学习！接下来可以：

1. 完成 **project_mnist_classifier.py** 项目
2. 进入 **4.2 RNN** 学习序列模型